In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_excel("alphaindex2000.xls")
# https://sites.google.com/site/jeffrlin/alphaindex2000.xls?attredirects=0
df.head()

,OCC_2000,SOC_CODE,OCCTITLE,INDRST2K,NAICSRST
0,896.0,51-9199,A mill operator,NaN,NaN
1,502.0,43-2021,A operator,668,51331
2,896.0,51-9199,A operator,Exc. 668,Exc. 51331
3,930.0,53-5011,A.B. seaman,#609,#483
4,593.0,43-9199,A.C.P. clerk,NaN,NaN


In [3]:
def clean_title(title):
    remove_word = ['of','the','and','or']
    cleaned = title.lower()
    cleaned = cleaned.replace('.','')
    cleaned = re.sub('[^a-z0-9]',' ',cleaned)
    cleaned_tokens = [w for w in re.split(' ',cleaned) if not w in remove_word]
    cleaned_tokens = sorted([w for w in cleaned_tokens if not w==''])
    return ' '.join(cleaned_tokens)

In [4]:
df['cleaned_title'] = df.OCCTITLE.apply(lambda x: clean_title(x))
df.head(20)

,OCC_2000,SOC_CODE,OCCTITLE,INDRST2K,NAICSRST,cleaned_title
0,896.0,51-9199,A mill operator,NaN,NaN,a mill operator
1,502.0,43-2021,A operator,668,51331,a operator
2,896.0,51-9199,A operator,Exc. 668,Exc. 51331,a operator
3,930.0,53-5011,A.B. seaman,#609,#483,ab seaman
4,593.0,43-9199,A.C.P. clerk,NaN,NaN,acp clerk
5,104.0,15-1041,A.D.P customer liaison,NaN,NaN,adp customer liaison
6,106.0,15-1061,A.D.P planner,NaN,NaN,adp planner
7,110.0,15-1071,A.D.P system coordinator,NaN,NaN,adp coordinator system
8,110.0,15-1071,A.D.P systems security,NaN,NaN,adp security systems
9,200.0,21-1019,A.S.A.T. C.O.R.E. counselor,NaN,NaN,asat core counselor


In [5]:
len(df)

30959

In [6]:
len(df.cleaned_title.unique())

26574

In [7]:
list_duplicating_titles = list()
list_cleaned_title = list()
list_cleaned_title2soc = list()
list_duplicates_with_soc = list()

for index, row in df.iterrows():
    original_title = str(row['OCCTITLE'])
    cleaned_title = str(row['cleaned_title'])
    soc = str(row['SOC_CODE'])
    cleaned_title2soc = cleaned_title + '\t' + soc
    
    if cleaned_title in list_cleaned_title:
        if cleaned_title2soc in list_cleaned_title2soc:
            pass
        else:
            list_duplicating_titles.append(cleaned_title)
    else:
        list_cleaned_title.append(cleaned_title)
        list_cleaned_title2soc.append(cleaned_title2soc)

list_duplicating_titles = sorted(list(set(list_duplicating_titles)))   
n_duplicating_titles = len(list_duplicating_titles)
print('Total duplicating titles = ' + str(n_duplicating_titles) )
        
for index, row in df.iterrows():
    original_title = row['OCCTITLE']
    cleaned_title = row['cleaned_title']
    onet = str(row['SOC_CODE'])
    
    if cleaned_title in list_duplicating_titles:
        loc = list_duplicating_titles.index(cleaned_title)
        output = [str(loc), cleaned_title, original_title, onet]
        list_duplicates_with_soc.append( '\t'.join(output) )

list_duplicates_with_soc = sorted(list_duplicates_with_soc)

duplicating_titles_file = open('duplicating_titles_2000.txt','w')
duplicating_titles_file.write( '\n'.join(list_duplicates_with_soc) )
duplicating_titles_file.close()

list_cleaned_title_file = open('cleaned_title_2000.txt','w')
list_cleaned_title_file.write( '\n'.join(list_cleaned_title) )
list_cleaned_title_file.close()

Total duplicating titles = 2501


In [8]:
list_duplicates_2digit = list()
list_duplicates_2digit_with_soc = list()
list_title2soc_2digit = list()
list_cleaned_title = list()

list_all_duplicates = open('duplicating_titles_2000.txt')

for line in list_all_duplicates:
    line_clean = re.sub('\n','',line)
    if not line == '':
        split = re.split('\t',line_clean)
        assert( len(split) == 4 )
        cleaned_title = split[1]
        original_title = split[2]
        soc2digit = str(re.sub('\D','',split[3]))[:2]
        title2soc_2digit = cleaned_title + '\t' + str(soc2digit)
        
        if cleaned_title in list_cleaned_title:
            if title2soc_2digit in list_title2soc_2digit:
                pass
            else:
                list_duplicates_2digit.append(cleaned_title)
        else:
            list_cleaned_title.append(cleaned_title)
            list_title2soc_2digit.append(title2soc_2digit)

list_duplicates_2digit = sorted(list(set(list_duplicates_2digit)))   
n_list_duplicates_2digit = len(list_duplicates_2digit)
print('Total duplicating titles (2 digits soc) = ' + str(n_list_duplicates_2digit) )

list_all_duplicates = open('duplicating_titles_2000.txt')

for line in list_all_duplicates:
    line_clean = re.sub('\n','',line)
    if not line == '':
        split = re.split('\t',line_clean)
        assert( len(split) == 4 )
        cleaned_title = split[1]
        original_title = split[2]
        soc = str(split[3])
        title2soc = cleaned_title + '\t' + original_title + '\t' + soc
        
        if cleaned_title in list_duplicates_2digit: 
            list_duplicates_2digit_with_soc.append(title2soc)
            
duplicating_title_2digit = open('duplicating_titles_2digit_2000.txt','w')
duplicating_title_2digit.write( '\n'.join(list_duplicates_2digit_with_soc) )
duplicating_title_2digit.close()

Total duplicating titles (2 digits soc) = 1318
